**2402. Meeting Rooms III**

**Hard**

**Companies**: Google

You are given an integer n. There are n rooms numbered from 0 to n - 1.

You are given a 2D integer array meetings where meetings[i] = [starti, endi] means that a meeting will be held during the **half-closed** time interval [starti, endi). All the values of starti are **unique**.

Meetings are allocated to rooms in the following manner:

- 1.Each meeting will take place in the unused room with the **lowest** number.

- 2.If there are no available rooms, the meeting will be delayed until a room becomes free. The delayed meeting should have the **same** duration as the original meeting.

- 3.When a room becomes unused, meetings that have an earlier original **start** time should be given the room.
  Return the number of the room that held the most meetings. If there are multiple rooms, return the room with the **lowest** number.

A **half-closed** interval [a, b) is the interval between a and b **including** a and **not including** b.

**Example 1:**

```
Input: n = 2, meetings = [[0,10],[1,5],[2,7],[3,4]]
Output: 0
```

**Explanation:**

- At time 0, both rooms are not being used. The first meeting starts in room 0.
- At time 1, only room 1 is not being used. The second meeting starts in room 1.
- At time 2, both rooms are being used. The third meeting is delayed.
- At time 3, both rooms are being used. The fourth meeting is delayed.
- At time 5, the meeting in room 1 finishes. The third meeting starts in room 1 for the time period [5,10).
- At time 10, the meetings in both rooms finish. The fourth meeting starts in room 0 for the time period [10,11).
  Both rooms 0 and 1 held 2 meetings, so we return 0.

**Example 2:**

```
Input: n = 3, meetings = [[1,20],[2,10],[3,5],[4,9],[6,8]]
Output: 1
```

**Explanation:**

- At time 1, all three rooms are not being used. The first meeting starts in room 0.
- At time 2, rooms 1 and 2 are not being used. The second meeting starts in room 1.
- At time 3, only room 2 is not being used. The third meeting starts in room 2.
- At time 4, all three rooms are being used. The fourth meeting is delayed.
- At time 5, the meeting in room 2 finishes. The fourth meeting starts in room 2 for the time period [5,10).
- At time 6, all three rooms are being used. The fifth meeting is delayed.
- At time 10, the meetings in rooms 1 and 2 finish. The fifth meeting starts in room 1 for the time period [10,12).
  Room 0 held 1 meeting while rooms 1 and 2 each held 2 meetings, so we return 1.

**Constraints:**

- 1 <= n <= 100
- 1 <= meetings.length <= 105
- meetings[i].length == 2
- 0 <= starti < endi <= 5 \* 105
- All the values of starti are unique.


In [ ]:
import heapq

class Solution:
    def mostBooked(self, n, meetings):
        """
        Algorithm (Greedy + Two Min-Heaps):

        We simulate the meeting allocation timeline using two priority queues.

        1) Sort all meetings by their original start time.
           → Ensures meetings are processed in correct priority order.

        2) Maintain TWO min-heaps:
           a) available:
              - Stores free room indices
              - Min-heap ensures we always pick the LOWEST numbered room

           b) busy:
              - Stores (endTime, roomIndex)
              - Min-heap ensures we always free the EARLIEST finishing room

        3) For each meeting [start, end]:
           a) First, release all rooms whose meetings finished on or before 'start'.
              - Because of half-open interval [a, b), a room finishing at 'start'
                can immediately be reused.

           b) If at least one room is available:
              - Assign the meeting immediately to the smallest room index.

           c) Otherwise (no room free):
              - Delay the meeting until the earliest finishing room becomes free.
              - Keep the SAME duration (end - start).
              - Assign it to that room.

           d) Increment the meeting count for the assigned room.

        4) After all meetings are processed:
           - Return the room with the highest meeting count.
           - If there is a tie, return the smallest room index.

        Time Complexity:
        - Sorting meetings: O(m log m)
        - Heap operations per meeting: O(log n)
        - Total: O(m log n)

        Space Complexity:
        - Heaps + counters: O(n)
        """

        # Step 1: Sort meetings by start time
        meetings.sort()

        # Step 2a: Min-heap of available rooms (room index)
        available = list(range(n))
        heapq.heapify(available)

        # Step 2b: Min-heap of busy rooms (endTime, roomIndex)
        busy = []

        # Step 3: Count how many meetings each room handles
        count = [0] * n

        # Step 4: Process each meeting
        for start, end in meetings:
            duration = end - start

            # Release rooms that have finished before current meeting starts
            while busy and busy[0][0] <= start:
                _, room = heapq.heappop(busy)
                heapq.heappush(available, room)

            # If a room is free, assign immediately
            if available:
                room = heapq.heappop(available)
                heapq.heappush(busy, (end, room))
            else:
                # Delay meeting to earliest finishing room
                finish_time, room = heapq.heappop(busy)
                heapq.heappush(busy, (finish_time + duration, room))

            # Record room usage
            count[room] += 1

        # Step 5: Return room with maximum usage (tie → lowest index)
        return count.index(max(count))
